# US Retail Insights - Hass Avocados

## Data Wrangling

In [1]:
from math import ceil
import pandas as pd
import datetime
import re

In [2]:
# Importing data set
raw_avocado = pd.read_csv('../Week_4/avocado.csv')

In [3]:
# Checking for nan
raw_avocado.isnull().mean()

Unnamed: 0      0.0
Date            0.0
AveragePrice    0.0
Total Volume    0.0
4046            0.0
4225            0.0
4770            0.0
Total Bags      0.0
Small Bags      0.0
Large Bags      0.0
XLarge Bags     0.0
type            0.0
year            0.0
region          0.0
dtype: float64

In [4]:
# Formating to date time
raw_avocado['date_formated'] = pd.to_datetime(raw_avocado['Date'])

In [5]:
# Checking which weekdays we have in the data frame
raw_avocado['weekday'] = pd.DatetimeIndex(raw_avocado['date_formated']).weekday
raw_avocado['weekday'].unique()

array([6])

In [6]:
# Calculating week of the month
def week_of_month(date):
    
    first_day = date.replace(day=1)
    dom = date.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

In [7]:
# Creating a new column and mapping week of the month function
raw_avocado['week_month'] = raw_avocado['date_formated'].map(week_of_month)

In [8]:
# Extracting month
raw_avocado['month'] = pd.DatetimeIndex(raw_avocado['date_formated']).month

In [9]:
# Concatenating year and month
raw_avocado['month_year'] = raw_avocado['year'].map(str) + ('_') + raw_avocado['month'].map(str)

In [10]:
# Moving annual total dictionary
moving_annual_dict = {
    '2015_12' : 'MAT1', '2015_11' : 'MAT1', '2015_10' : 'MAT1', 
    '2015_9' : 'MAT1', '2015_8' : 'MAT1', '2015_7' : 'MAT1',
    '2015_6' : 'MAT1', '2015_5' : 'MAT1', '2015_4' : 'MAT1', 
    '2015_3' : 'OUT', '2015_2' : 'OUT', '2015_1': 'OUT',
    '2016_12' : 'MAT2', '2016_11' : 'MAT2', '2016_10' : 'MAT2',
    '2016_9' : 'MAT2', '2016_8' : 'MAT2', '2016_7' :'MAT2',
    '2016_6' : 'MAT2', '2016_5' : 'MAT2', '2016_4' : 'MAT2', 
    '2016_3' : 'MAT1', '2016_2' : 'MAT1', '2016_1' : 'MAT1',
    '2017_12' : 'MAT3', '2017_11' : 'MAT3', '2017_10' : 'MAT3', 
    '2017_9' : 'MAT3', '2017_8' : 'MAT3', '2017_7' : 'MAT3',
    '2017_6' : 'MAT3', '2017_5' : 'MAT3', '2017_4' : 'MAT3', 
    '2017_3' : 'MAT2', '2017_2' : 'MAT2', '2017_1' : 'MAT2',
    '2018_3' : 'MAT3', '2018_2' : 'MAT3', '2018_1' : 'MAT3'
    }

In [11]:
# Crating a new column with moving annual total values
raw_avocado['moving_annual'] = raw_avocado['month_year'].map(moving_annual_dict)

In [12]:
raw_avocado = raw_avocado.rename(columns = {
                                'Date' : 'raw_date',
                                'date_formated' : 'date',
                                'AveragePrice' : 'average_price',
                                'Total Volume' : 'total_volume',
                                '4046' : 'var_4046',
                                '4225' : 'var_4225',
                                '4770' : 'var_4770',
                                'Total Bags' : 'total_bags',
                                'Small Bags' : 'small_bags',
                                'Large Bags' : 'large_bags',
                                'XLarge Bags' : 'xlarge_bags',
                                })

In [13]:
raw_avocado['city_region'] = raw_avocado['region']\
                                .apply(lambda region: (' ').join(re.findall('[A-Z][^A-Z]*', region)))

In [14]:
city_dict = {
    'Baltimore Washington' : 'Wachington, D.C.', 
    'Buffalo Rochester' : 'Buffalo', 
    'Cincinnati Dayton' : 'Cincinnati', 
    'Dallas Ft Worth' : 'Dallas',
    'Harrisburg Scranton' : 'Scranton',
    'Hartford Springfield' : 'Springfield',
    'Miami Ft Lauderdale' : 'Miami',
    'New Orleans Mobile' : 'New Orleans',
    'Phoenix Tucson' : 'Phoenix',
    'Raleigh Greensboro' : 'Raleigh',
    'Richmond Norfolk' : 'Norfolk'
    }

In [15]:
raw_avocado['city_plot'] = raw_avocado['city_region'].map(city_dict)\
                                .fillna(raw_avocado['city_region'])

In [16]:
city_region_dict = {
    'Baltimore Washington' : 'Baltimore/Washington Area', 
    'Buffalo Rochester' : 'Buffalo/Rochester Area', 
    'Cincinnati Dayton' : 'Cincinnati/Dayton Area', 
    'Dallas Ft Worth' : 'Dallas/Ft.Worth Area',
    'Harrisburg Scranton' : 'Harrisburg/Scranton Area',
    'Hartford Springfield' : 'Hartford/Springfield Area',
    'Miami Ft Lauderdale' : 'Miami/Ft.Lauderdale Area',
    'New Orleans Mobile' : 'New Orleans/Mobile Area',
    'Phoenix Tucson' : 'Phoenix/Tucson Area',
    'Raleigh Greensboro' : 'Raleigh/Greensboro Area',
    'Richmond Norfolk' : 'Richmond/Norfolk Area'
    }

In [17]:
raw_avocado['city_label'] = raw_avocado['city_region'].map(city_region_dict)\
                                .fillna(raw_avocado['city_region'])

In [18]:
macroreg_flag = [
    'California',       
    'Great Lakes', 
    'Midsouth', 
    'Northeast',  
    'Northern New England', 
    'Plains', 
    'South Carolina',   
    'South Central', 
    'Southeast',
    'Total U S', 
    'West', 
    'West Tex New Mexico'
    ]

In [19]:
raw_avocado['macroreg_flag'] = raw_avocado['city_region']\
                                .apply(lambda city: city in macroreg_flag)

In [20]:
price_buckets_conv = raw_avocado.query('city_region == "Total U S" and type == "conventional" and moving_annual == "MAT3"')['average_price'].describe()
price_buckets_org = raw_avocado.query('city_region == "Total U S" and type == "organic" and moving_annual == "MAT3"')['average_price'].describe()

In [21]:
def price_bucket_conv(i):
    if i >= 0.87 and i < 0.97:
        i = 'bucket1'
    elif i >= 0.97 and i < 1.07:
        i = 'bucket2'
    elif i >= 1.07 and i < 1.17:
        i = 'bucket3'
    elif i >= 1.17 and i < 1.27:
        i = 'bucket4'
    elif i >= 1.27 and i < 1.37:
        i = 'bucket5'
    elif i >= 1.37 and i < 1.47:
        i = 'bucket6'
    elif i >= 1.47 and i < 1.57:
        i = 'bucket7'
    elif i >= 1.57 and i <= 1.65:
        i = 'bucket8'
    else:
        i = 'outlier'
    return i

In [22]:
def price_bucket_org(i):
    if i >= 1.42 and i < 1.52:
        i = 'bucket1'
    elif i >= 1.52 and i < 1.62:
        i = 'bucket2'
    elif i >= 1.62 and i < 1.72:
        i = 'bucket3'
    elif i >= 1.72 and i < 1.82:
        i = 'bucket4'
    elif i >= 1.82 and i < 1.92:
        i = 'bucket5'
    elif i >= 1.92 and i < 2.02:
        i = 'bucket6'
    elif i >= 2.02 and i <= 2.09:
        i = 'bucket7'
    else:
        i = 'outlier'
    return i

In [23]:
raw_avocado['price_bucket_conv'] = raw_avocado.loc[raw_avocado['type']=='conventional']['average_price']\
                                        .apply(price_bucket_conv)

In [24]:
raw_avocado['price_bucket_org'] = raw_avocado.loc[raw_avocado['type']=='organic']['average_price']\
                                    .apply(price_bucket_org)

In [25]:
# Number of supermarket store in the US from 2011 to 2018 (source: statista)
store_number = {2015 : '38015', 2016 : '38441', 2017 : '38571', 2018 : '38307'}

In [26]:
raw_avocado['number_of_stores'] = raw_avocado.loc[raw_avocado['city_region'] == 'Total U S']['year'].map(store_number)

In [27]:
avocado = raw_avocado[[
    'year',
    'raw_date',
    'date',
    'weekday',
    'week_month',
    'moving_annual',
    'average_price',
    'price_bucket_conv',
    'price_bucket_org',
    'total_volume',                      
    'var_4046',                    
    'var_4225',                    
    'var_4770',                      
    'total_bags',                     
    'small_bags',                     
    'large_bags',                    
    'xlarge_bags',                    
    'type',                                          
    'region',
    'city_region',
    'city_plot',
    'city_label',
    'macroreg_flag',
    'number_of_stores'
]]

In [28]:
#avocado.to_csv('avocado_worked.csv', index=True)

In [29]:
avocado.head(2)

,year,raw_date,date,weekday,week_month,moving_annual,average_price,price_bucket_conv,price_bucket_org,total_volume,...,small_bags,large_bags,xlarge_bags,type,region,city_region,city_plot,city_label,macroreg_flag,number_of_stores
0,2015,2015-12-27,2015-12-27,6,4,MAT1,1.33,bucket5,NaN,64236.62,...,8603.62,93.25,0.0,conventional,Albany,Albany,Albany,Albany,False,NaN
1,2015,2015-12-20,2015-12-20,6,3,MAT1,1.35,bucket5,NaN,54876.98,...,9408.07,97.49,0.0,conventional,Albany,Albany,Albany,Albany,False,NaN


In [56]:
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
stats = avocado.query('city_region == "Total U S" & moving_annual == "MAT3" & type == "conventional"')['average_price'].describe()

In [72]:
stats['IQR'] = stats['75%'] - stats['25%']
stats

count    52.000000
mean      1.235577
std       0.185867
min       0.870000
25%       1.090000
50%       1.195000
75%       1.322500
max       1.650000
IQR       0.232500
Name: average_price, dtype: float64

In [75]:
iqr = stats['IQR']
q3 = stats['75%']
q1 = stats['25%']

In [76]:
upper_limit = q3 + 1.5 * iqr
lower_limit = q1 - 1.5 * iqr

In [77]:
upper_limit

1.67125

In [78]:
lower_limit

0.7412500000000002

In [84]:
stats_org = avocado.query('city_region == "Total U S" & moving_annual == "MAT3" & type == "organic"')['average_price'].describe()

In [85]:
stats_org['IQR'] = stats_org['75%'] - stats_org['25%']
stats_org

count    52.000000
mean      1.700192
std       0.171069
min       1.420000
25%       1.557500
50%       1.685000
75%       1.820000
max       2.090000
IQR       0.262500
Name: average_price, dtype: float64

In [86]:
iqr_org = stats_org['IQR']
q3_org = stats_org['75%']
q1_org = stats_org['25%']

In [87]:
upper_limit_org = q3 + 1.5 * iqr
lower_limit_org = q1 - 1.5 * iqr

In [88]:
upper_limit_org

1.67125

In [89]:
lower_limit_org 

0.7412500000000002